In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

image_path = 'C:/Users/tabm9/OneDrive - Caissa Analytica/Documents/DS_Portfolio/Projects/Images/RLCS/'

In [368]:
def bound_array(a, bounds=[0, 1]):
    s = a.copy()

    # Check number of times it breaks bounds
    signed_crosses = np.floor((s - bounds[0]) / (bounds[1] - bounds[0])).astype(int)
    crosses = np.floor(np.where(signed_crosses < 0, -signed_crosses, signed_crosses)).astype(int)

    # Reorient direction
    s *= ((-1) ** crosses)

    # Reposition to maintain continuity
    s += (bounds[1] - bounds[0]) * signed_crosses * (-1) ** (crosses + 1)
    s += (bounds[1] + bounds[0]) * 0 ** ((crosses + 1) % 2)

    return s


def bounded_random_walk(initial=1/2, bounds=[0, 1], size=[1000, 10], drift=0, std=0.01, is_deterministic=False):
    if is_deterministic: return np.ones(size) * initial

    # Generate random walk initialized on 0
    walk = np.random.normal(loc=drift, scale=std, size=size).cumsum(0) + initial

    # Adjust walk to bounds
    walk = bound_array(walk, bounds)        

    return walk

fig = go.Figure()

walks = bounded_random_walk().T

for walk in walks:
    fig.add_trace(go.Scatter(y=walk, opacity=0.5, showlegend=False))

fig.show()

# Test with only 2 teams

In [347]:
bounds = [-1.5, -1]

a = np.array(range(100)) / 10
a_orig = a.copy()

# Check number of times it breaks bounds
signed_crosses = np.floor((a - bounds[0]) / (bounds[1] - bounds[0])).astype(int)
crosses = np.floor(np.where(signed_crosses < 0, -signed_crosses, signed_crosses)).astype(int)

# Reorient direction
a *= ((-1) ** crosses)

# Reposition to maintain continuity
a += (bounds[1] - bounds[0]) * signed_crosses * (-1) ** (crosses + 1)
a += (bounds[1] + bounds[0] ) * 0 ** ((crosses + 1) % 2)

fig = go.Figure()

fig.add_trace(go.Scatter(y=a))
fig.add_trace(go.Scatter(y=a_orig))
fig.add_trace(go.Scatter(y=[bounds[0]] * len(a), line=dict(color='black')))
fig.add_trace(go.Scatter(y=[bounds[1]] * len(a), line=dict(color='black')))

fig.show()

## Naive Elo

In [2]:
def run_games(Pa=1/4, K=1, N=[10000, 1], thres=0.10, stop_burnout=False):
    # Run N games
    Sa = (np.random.random(N) < Pa).astype(int)

    # Calculate Ratings
    Ra = np.zeros(N)
    pa = np.zeros(N)
    pa[1] += 1/2
    finished_burnout = np.array([False] * N[1])
    n_to_convergence = np.ones(N[1]) * (N[0] + 1)
    for i in range(1, N[0]):
        # Update rating
        Ra[i] = Ra[i-1] + K * (Sa[i - 1] - pa[i - 1])

        # Update Approx Probabilities
        pa[i] = 1 / (1 + 10 ** (-2 * Ra[i] / 400))

        # Check if burnout is finished
        if not all(finished_burnout):
            condition = abs(pa[i] - Pa) <= Pa * thres
            finished_burnout = np.where(condition, True, finished_burnout)
            n_to_convergence = np.where(finished_burnout, n_to_convergence, i + 1)

            if all(finished_burnout) and stop_burnout:
                break

    return pa.T, n_to_convergence
        

## Improved Elo

In [3]:
def logbase(base, x):
    return np.log(x) / np.log(base)

def K_generator(K_interval=[0,400], K_default=300):
    K_d = (K_default - K_interval[0]) / (K_interval[1] - K_interval[0])
    b = logbase(1/2, 1/2 - np.log((1 + np.e ** (-2 * np.e) - K_d) / (K_d + np.e ** (-2 * np.e))) / (4 * np.e))

    def K(x, b=b, K_interval=K_interval):
        return (K_interval[1] - K_interval[0]) * ((1 + 2 * np.e ** (-2 * np.e)) / (1 + np.e ** (-4 * np.e * (x ** b - 1/2))) - np.e ** (-2 * np.e)) + K_interval[0]

    return K



def run_games_improved(Pa=1/4, K_interval=[0,200], K_default=180, N=[10000, 1], thres=0.10, thres_n=10, stop_burnout=False):
    # Run N games
    Sa = (np.random.random(N) < Pa).astype(int)

    # Calculate Ratings
    Ra = np.zeros(N)
    pa = np.zeros(N)
    pa[1] += 1/2
    fpa_list = pa.copy()
    finished_burnout = np.array([False] * N[1])
    n_to_convergence = np.ones(N[1]) * (N[0] + 1)
    K = K_generator(K_interval=K_interval, K_default=K_default)
    K_list = -np.ones(N)

    for i in range(1, N[0]):
        # Update rating
        frequentist_pa = Sa[:i].sum(0) / i
        
        K_i = K(abs(pa[i - 1] - frequentist_pa))
        Ra[i] = Ra[i-1] + K_i * (Sa[i - 1] - pa[i - 1])

        K_list[i] = K_i
        fpa_list[i] = frequentist_pa

        # Update Approx Probabilities
        pa[i] = 1 / (1 + 10 ** (-2 * Ra[i] / 400))
        # print(K_i, Ra[i], pa[i])

        # Check if burnout is finished
        if not all(finished_burnout) and i >= thres_n:
            condition = (abs(pa[i - thres_n: i] - Pa) <= Pa * thres).T.all(1)
            finished_burnout = np.where(condition, True, finished_burnout)
            n_to_convergence = np.where(finished_burnout, n_to_convergence, i + 1 - thres_n)

            if all(finished_burnout) and stop_burnout:
                break

    return pa.T, n_to_convergence, fpa_list.T

## Comparison

In [22]:
# Setup parameters
Pa = 1/4
N_pa = [3000, 100]
N_conv = [3000, 10000]
N_range_pa = list(range(N_pa[0] + 1))

# Simulation
pa, _ = run_games(Pa=Pa, N=N_pa)
pa_improved, _, _ = run_games_improved(Pa=Pa, N=N_pa)

_, n_to_convergence = run_games(Pa=Pa, N=N_conv)
_, n_to_convergence_improved, fpa_list = run_games_improved(Pa=Pa, N=N_conv)


# Initialize plot
fig = make_subplots(
    rows = 2, 
    subplot_titles=('Actual vs. Approximated Probabilities', 'Time to Convergence Distributions')
)

# Add Probability lines
fig.add_trace(go.Scatter(x=[np.nan], y=[np.nan], line=dict(color='blue'), mode='lines', name='Approximated Probability'), row=1, col=1)
fig.add_trace(go.Scatter(x=[np.nan], y=[np.nan], line=dict(color='red'), mode='lines', name='Improved Approximated Probability'), row=1, col=1)

for i in range(N_pa[1]):
    fig.add_trace(go.Scatter(x=N_range_pa, y=pa[i], opacity=np.sqrt(1 / N_pa[1]), line=dict(color='blue'), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=N_range_pa, y=pa_improved[i], opacity=np.sqrt(1 / N_pa[1]), line=dict(color='red'), showlegend=False), row=1, col=1)

fig.add_trace(go.Scatter(x=N_range_pa, y=[Pa] * (N_pa[0] + 1), line=dict(color='light green'), name='Real Probability'), row=1, col=1)

# Prepare histogram parameters
bin_width = 50
bins = list(range(0, int(np.ceil(max(max(n_to_convergence), max(n_to_convergence_improved)))), bin_width))

x_naive = np.mean(n_to_convergence)
y_naive = sum(np.digitize(n_to_convergence, bins) == np.digitize(x_naive, bins)) / N_conv[1]
x_improved = np.mean(n_to_convergence_improved)
y_improved = sum(np.digitize(n_to_convergence_improved, bins) == np.digitize(x_improved, bins)) / N_conv[1]

# Add histograms
fig.add_trace(go.Histogram(histnorm='probability', x=n_to_convergence, xbins=dict(size=bin_width), opacity=0.5, showlegend=False), row=2, col=1)
fig.add_trace(go.Histogram(histnorm='probability', x=n_to_convergence_improved, xbins=dict(size=bin_width), opacity=0.5, showlegend=False), row=2, col=1)

fig.add_annotation(x=x_naive, y=y_naive, xref='x2', yref='y2', text='Naive Elo', ax=50)
fig.add_annotation(x=x_improved, y=y_improved, xref='x2', yref='y2', text='Improved Elo', ax=50)

# Update Layout
fig.update_layout(barmode='overlay', title='Test with 2 Teams, Static Probabilities')
fig.update_xaxes(title_text='Iteration', row=1, col=1)
fig.update_xaxes(title_text='Iteration', row=2, col=1)
fig.update_yaxes(title_text='Probability', range=[0, 1], row=1, col=1)
fig.update_yaxes(title_text='Probability', row=2, col=1)

# Show / save figure
fig.show(renderer='browser')
fig.write_html(image_path + '2_Teams_Static_Probability.html')